In [1]:
import os
import json
import pandas as pd
from sqlalchemy import text
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import gensim

import sys
from nltk.tokenize import sent_tokenize

from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import ast
from nltk.corpus import wordnet

In [2]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jengyuchou/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
normalized_dir = "./polarities/lemmatized_normalized"

In [4]:
len(os.listdir(normalized_dir))

19

In [5]:
subreddits = list()

In [6]:
subreddit_polarities = dict()
for subreddit in os.listdir(normalized_dir):
    if subreddit.split('_standardized')[0] != 'BingeEatingRecovery' and subreddit.split('_standardized')[0] != 'selftext':
        with open(os.path.join(normalized_dir,subreddit)) as f:
            subreddit_polarities[subreddit.split('_standardized')[0]] = json.load(f)
            subreddits.append(subreddit.split('_standardized')[0])

In [7]:
subreddit_polarities.keys()

dict_keys(['AnorexiaRecovery', 'bingeeating', 'eating_disorders', 'EdAnonymousAdults', 'EatingDisorderHope', 'bulimia', 'BulimiaRecovery', 'EDRecovery_public', 'BingeEatingDisorder', 'EatingDisorders', 'anorexiaflareuphelp', 'edsupport', 'PurgingDisorder', 'AnorexiaNervosa', 'BulimiaAndAnaSupport', 'EDAnonymous', 'NotOtherwiseSpecified'])

In [8]:
subreddits

['AnorexiaRecovery',
 'bingeeating',
 'eating_disorders',
 'EdAnonymousAdults',
 'EatingDisorderHope',
 'bulimia',
 'BulimiaRecovery',
 'EDRecovery_public',
 'BingeEatingDisorder',
 'EatingDisorders',
 'anorexiaflareuphelp',
 'edsupport',
 'PurgingDisorder',
 'AnorexiaNervosa',
 'BulimiaAndAnaSupport',
 'EDAnonymous',
 'NotOtherwiseSpecified']

In [9]:
sqlite_path = "./reddit_ed_0.6.0.sqlite3"
engine = create_engine(
    f"sqlite:///{sqlite_path}",
    connect_args={"check_same_thread":False},
)

In [10]:
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [11]:
def lemmatized_preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(lemmatize_sentence(text)) :
        
        if token not in stopwords.words('english'):
            
            result.append(token)
    return result

In [12]:
lemmatizer = WordNetLemmatizer()

# Averaging Polarity Scores

In [13]:
polarity_dfs = dict()
for subreddit in subreddits:
    if subreddit != 'BingeEatingRecovery' and subreddit != 'selftext':
        polarities = list()
        print(subreddit)
        query = f"select id, selftext from subreddit_submission_metadata where subreddit='{subreddit}';"
        df = pd.read_sql_query(text(query),con=engine.connect())
        for sentence in df.values:
            preprocessed_sentence = lemmatized_preprocess(sentence[1])
            sentence_polarities = [subreddit_polarities[subreddit][word] for word in preprocessed_sentence if word in subreddit_polarities[subreddit].keys()]
        #     print(sentence_polarities)
            avg_polarities = np.average(sentence_polarities)
        #     print(f"Regular sum of sentence non-stopwords polarities: {sum_polarities}")  
        #     print(sentence.append(sum_polarities))
            polarities.append({'id':sentence[0], 'selftext':sentence[1],'sentence_polarity':avg_polarities})


        polarity_dfs[subreddit] = pd.DataFrame.from_dict(polarities)

AnorexiaRecovery


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


bingeeating
eating_disorders


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


EdAnonymousAdults


...

BulimiaAndAnaSupport


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


EDAnonymous


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NotOtherwiseSpecified


In [14]:
polarity_dfs.keys()

dict_keys(['AnorexiaRecovery', 'bingeeating', 'eating_disorders', 'EdAnonymousAdults', 'EatingDisorderHope', 'bulimia', 'BulimiaRecovery', 'EDRecovery_public', 'BingeEatingDisorder', 'EatingDisorders', 'anorexiaflareuphelp', 'edsupport', 'PurgingDisorder', 'AnorexiaNervosa', 'BulimiaAndAnaSupport', 'EDAnonymous', 'NotOtherwiseSpecified'])

In [15]:
for subreddit in polarity_dfs.keys():
#     print(subreddit)
    sub = [subreddit for i in range(len(polarity_dfs[subreddit]))]
    polarity_dfs[subreddit]['subreddit'] = sub

In [16]:
all_subreddits_polarity_df = pd.concat([polarity_dfs[subreddit] for subreddit in polarity_dfs.keys()])

In [17]:
all_subreddits_polarity_df.sort_values(by='sentence_polarity')

,id,selftext,sentence_polarity,subreddit
584,znaf5q,i hate myself,-12.241338,AnorexiaNervosa
42,10o9fkb,I hate it,-10.214032,BingeEatingDisorder
390,10mmavk,That’s it. That’s the post.,-8.334615,EDAnonymous
282,10nisvw,That's it that's the post,-8.334615,EDAnonymous
1051,joxgmd,That’s it that’s the post,-7.649268,eating_disorders
...,...,...,...,...
1967,10ajqjq,💖,NaN,EDAnonymous
3065,101sqec,.,NaN,EDAnonymous
3316,zznufi,👨‍🦲💕,NaN,EDAnonymous
3368,zz93po,😀,NaN,EDAnonymous


## Combined Sentence Polarity Scores for All Individual Subreddits Excluding BingeEatingRecovery

In [18]:
all_subreddits_polarity_df.groupby(by='subreddit')['id'].count()

subreddit
AnorexiaNervosa           863
AnorexiaRecovery         2020
BingeEatingDisorder       368
BulimiaAndAnaSupport       76
BulimiaRecovery            13
EDAnonymous              3774
EDRecovery_public         361
EatingDisorderHope         62
EatingDisorders           893
EdAnonymousAdults        3530
NotOtherwiseSpecified       8
PurgingDisorder            12
anorexiaflareuphelp        33
bingeeating               252
bulimia                  1901
eating_disorders         2440
edsupport                  63
Name: id, dtype: int64

In [19]:
all_subreddits_polarity_df.to_csv('submission_polarity.csv')